In [2]:
# Full simulation code for CR dispatching strategy
# Includes:
# - Balanced sampling
# - EHL prioritization
# - Setup times
# - Tool downtimes
# - Dedicated tools for EHL (via priority)

import simpy
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns

# --- Data Loading and Preprocessing ---
file_path = "SMT_2020_Model_Data_-_LVHM_E.xlsx"
lot_df = pd.read_excel(file_path, sheet_name="Lotrelease - variable due dates")
lot_df_eng = pd.read_excel(file_path, sheet_name="Lotrelease - Engineering")
route_df = pd.read_excel(file_path, sheet_name='Route_Product_4')

# Combine and preprocess lot data
lot_df = pd.concat([lot_df, lot_df_eng], ignore_index=True)
lot_df['START DATE'] = pd.to_datetime(lot_df['START DATE'])
lot_df['DUE DATE'] = pd.to_datetime(lot_df['DUE DATE'])
base_time = lot_df['START DATE'].min()
lot_df['ReleaseTimeMin'] = (lot_df['START DATE'] - base_time).dt.total_seconds() / 60
lot_df['DueTimeMin'] = (lot_df['DUE DATE'] - base_time).dt.total_seconds() / 60

# Route steps
steps = route_df[['STEP', 'TOOLGROUP', 'MEAN']].dropna()
steps['MEAN'] = pd.to_numeric(steps['MEAN'], errors='coerce')

# --- Constants and Configuration ---
SIM_TIME = 1100000 # Total simulation time
SETUP_TIME = 10     # Time for tool setup change
PM_INTERVAL = 60000 # Preventive Maintenance interval
PM_DURATION = 500   # Preventive Maintenance duration
MTBF = 100000        # Mean Time Between Failures
MTTR = 2000         # Mean Time To Repair

# --- Lot Classification and Sampling ---
def classify_lot_type(lot_id):
    if pd.isna(lot_id): return "Unknown"
    lot_str = str(lot_id).upper()
    if "ENG" in lot_str:
        return "EHL" if "HL" in lot_str or "HOT" in lot_str else "ERL"
    elif "HL" in lot_str or "HOT" in lot_str:
        return "PHL"
    else:
        return "PRL"

def balanced_sample(df, n_each=100):
    sampled = []
    for t in ['PRL', 'PHL', 'ERL', 'EHL']:
        subset = df[df['LOT NAME/TYPE'].apply(classify_lot_type) == t]
        if not subset.empty:
            sampled.append(subset.sample(n=min(n_each, len(subset)), random_state=42))
    return pd.concat(sampled)

# Apply classification and due date assignment
lot_df = balanced_sample(lot_df, n_each=100)
lot_df['LotType'] = lot_df['LOT NAME/TYPE'].apply(classify_lot_type)

# Explicitly set due dates for consistency and to show buffer
# IdealCycleTime for all lots (sum of means) is ~16115.058 minutes
# Current due dates provide significant buffer:
# EHL/PHL: 30000 min buffer (30000 - 16115.058 = ~13885 min additional buffer)
# ERL/PRL: 70000 min buffer (70000 - 16115.058 = ~53885 min additional buffer)
lot_df.loc[lot_df['LotType'].isin(['PHL', 'EHL']), 'DueTimeMin'] = lot_df['ReleaseTimeMin'] + 30000
lot_df.loc[lot_df['LotType'].isin(['PRL', 'ERL']), 'DueTimeMin'] = lot_df['ReleaseTimeMin'] + 70000


# --- Simulation Logic ---

def run_cr_dispatch_sim():
    env = simpy.Environment()
    
    # Initialize tool resources
    tool_resources = {
        tool: simpy.PriorityResource(env, capacity=2) # Assuming tools have capacity 2
        for tool in steps['TOOLGROUP'].unique()
    }
    
    last_operation = {tool: None for tool in steps['TOOLGROUP'].unique()} # To track setup times

    # --- Tool Downtime Behavior ---
    def tool_downtime_behavior(env, tool_name, tool_resource):
        while True:
            # Preventive Maintenance
            yield env.timeout(PM_INTERVAL)
            with tool_resource.request(priority=-1000) as req: # PM gets very high priority
                yield req
                yield env.timeout(PM_DURATION)

            # Random Failure
            yield env.timeout(random.expovariate(1.0 / MTBF))
            with tool_resource.request(priority=-2000) as req: # Breakdowns get even higher priority
                yield req
                yield env.timeout(random.expovariate(1.0 / MTTR))

    # Start downtime processes for all tools
    for tool_name, resource in tool_resources.items():
        env.process(tool_downtime_behavior(env, tool_name, resource))

    lot_records = {} # To store results for each lot

    # --- Lot Processing Logic ---
    def process_lot(env, lot_id, release, due, lot_type):
        yield env.timeout(max(0, release - env.now)) # Wait until lot is released
        
        # Store initial record for the lot
        lot_records[lot_id] = {
            'start': env.now,
            'due': due,
            'type': lot_type,
            'release': release # Store release time to calculate target cycle time later
        }
        
        # Calculate theoretical minimum processing time for this lot's route
        min_processing_time = steps['MEAN'].sum() 
        lot_records[lot_id]['min_processing_time'] = min_processing_time
        
        for idx, row in steps.iterrows():
            tool = row['TOOLGROUP']
            proc_time = row['MEAN']

            # Calculate Remaining Processing Time (RPT) from the current step onwards
            remaining_proc_time = steps.loc[idx:, 'MEAN'].sum() 
            
            time_until_due = max(due - env.now, 0.01) # Avoid division by zero
            
            # CR calculation: Lower CR is more critical
            cr = remaining_proc_time / time_until_due
            
            # EHL prioritization: Make EHLs even more critical (lower CR)
            if lot_type == 'EHL':
                cr *= 0.1 # Make EHL CR even smaller to give higher priority
            
            # SimPy Priority: Lower value = higher priority. So, priority = CR
            priority = cr 

            with tool_resources[tool].request(priority=priority) as req:
                yield req # Wait for tool to become available

                # Setup time logic
                if last_operation[tool] != row['STEP']:
                    yield env.timeout(SETUP_TIME)
                    last_operation[tool] = row['STEP']
                
                yield env.timeout(proc_time) # Process the lot

        lot_records[lot_id]['end'] = env.now # Mark end time when all steps are completed

    # --- Lot Release Process ---
    def lot_release(env):
        for _, row in lot_df.iterrows():
            lot_id = row['LOT NAME/TYPE']
            release = row['ReleaseTimeMin']
            due = row['DueTimeMin']
            lot_type = classify_lot_type(lot_id) # Re-classify to ensure consistency
            env.process(process_lot(env, lot_id, release, due, lot_type))
            yield env.timeout(0) # Allow other events to be scheduled

    # Run the simulation
    env.process(lot_release(env))
    env.run(until=SIM_TIME)

    # --- Collect and Return Results ---
    rows = []
    for lot_id, rec in lot_records.items():
        if 'end' in rec: # Only include completed lots
            start = rec['start']
            end = rec['end']
            due = rec['due']
            release = rec['release']
            
            cycle = end - start
            lateness = end - due
            min_processing_time = rec['min_processing_time']
            # Target Cycle Time: The total time allowed from release to due date
            target_cycle_time = due - release
            
            rows.append({
                'Lot': lot_id,
                'Type': rec['type'],
                'Start': start,
                'End': end,
                'Due': due,
                'CycleTime': cycle,
                'MinProcessingTime': min_processing_time, # Pure processing time
                'TargetCycleTime': target_cycle_time,     # Ideal + Buffer
                'Lateness': lateness,
                'OnTime': end <= due
            })

    return pd.DataFrame(rows)

# --- Run and Summarize Simulation ---
df_cr = run_cr_dispatch_sim()
df_cr['Group'] = df_cr['Type'] # Keep 'Group' for consistent plotting



In [4]:

plt.figure(figsize=(10, 6))
sns.boxplot(
    data=df_cr,
    x='Group',
    y='CycleTime',
    palette='pastel',  # Softer, cleaner look
    width=0.5,         # Narrower boxes
    fliersize=3        # Smaller outliers
)

# Visual tuning
plt.ylim(18500, 19600)  # Adjust this range based on your data
plt.title("Cycle Time by Lot Type (CR Strategy with Setup + Downtime)", fontsize=14, weight='bold')
plt.xlabel("Lot Type", fontsize=12)
plt.ylabel("Cycle Time (minutes)", fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Tight layout to eliminate excess white space
plt.tight_layout(pad=1.0)
plt.subplots_adjust(bottom=0.15)  # Optional: fine-tune spacing below

plt.show()


NameError: name 'compare_df' is not defined